In [1]:
import copy
from difflib import SequenceMatcher
import os
import pandas as pd
from tqdm import tqdm

<h2>Cargamos libro Excel</h2>

In [2]:
df_main = pd.read_excel('Compilado02.xlsx', engine='openpyxl', sheet_name=None)

for key in df_main.keys():
    print(key)
    

alumnas2020
alumnas2021
clases


<h2>Hacemos las comparaciones de nombres</h2>

In [3]:
df_students01 = df_main['alumnas2020'].copy(deep=True)
df_students02 = df_main['alumnas2021'].copy(deep=True)

# This function will help us to compare strings
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

# This one makes the comparison industrial
def gen_similar(list01, list02):
    list_similars = []
    
    for word in list01:
        ratio = 0

        for spiegel in list02:
            if ratio < max(ratio, similar(word, spiegel)):
                ratio = similar(word, spiegel)
                solution = (word, ratio, spiegel)

        list_similars.append(solution)

    return(list_similars)


list01 = df_students01['Nombre'].to_list()
list02 = df_students02['Nombre'].to_list()
list_similars = gen_similar(list01, list02)

<h2>Creamos nuestra columna relacional</h2>

In [5]:
for row in range(df_students02.shape[0]):
    name_2021 = df_students02.at[row, 'Nombre']
    
    for i, col in enumerate(['Nombre 2020', 'Name match %']):
        value = [entry[i] for entry in list_similars if entry[-1] == name_2021]

        if len(value) != 0:
            df_students02.at[row, col] = value[0]
            
dict_final = {'students2020': df_students01, 'students2021': df_students02, 'classes': df_main['clases']}

# Adicionalmente, se guarda el diccionario en formato de libro en Excel
writer = pd.ExcelWriter('test.xlsx', engine='xlsxwriter')

for df_name, df in dict_final.items():
    df.to_excel(writer, sheet_name=df_name)
    
writer.save()

In [21]:
num = 289
den = 2

while den < 100:
    if num % den == 0:
        print(den, num/den)
        break
    
    den += 1

17 17.0


<h2>Agregamos los nombres del 2021 como llave</h2>

In [6]:
names_2020 = dict_final['students2020']['Nombre'].to_list()
names_2021 = dict_final['students2021']['Nombre'].to_list()

df_2020 = dict_final['students2021'].copy(deep=True)
df_2021 = dict_final['students2021'].copy(deep=True)
df_classes = dict_final['classes'].copy(deep=True)

# We create our new columns with None values for every row
df_classes['name 2020'], df_classes['ratio 2020'], df_classes['email 2020'] = None, None, None
df_classes['name 2021'], df_classes['ratio 2021'], df_classes['email 2021'] = None, None, None

# A list called "cols" is created to order our columns in the future
cols = [col for col in df_classes]

# Through an iteration the best match-ratio gets chosen
for i, row in enumerate(tqdm(range(df_classes.shape[0]))):
    name = df_classes.at[row, 'Nombre']
    solution_2020, solution_2021 = None, None
    ratio_2020, ratio_2021 = 0, 0

    for name_2020 in names_2020:
        if ratio_2020 < max(ratio_2020, similar(name, name_2020)):
            ratio_2020 = similar(name, name_2020)
            solution_2020 = name_2020
    
    for name_2021 in names_2021:
        if ratio_2021 < max(ratio_2021, similar(name, name_2021)):
            ratio_2021 = similar(name, name_2021)
            solution_2021 = name_2021
    
    # In this line we finally fill our rows with the corresponding information
    df_classes.at[row, 'name 2020'], df_classes.at[row, 'ratio 2020'] = solution_2020, ratio_2020
    df_classes.at[row, 'name 2021'], df_classes.at[row, 'ratio 2021'] = solution_2021, ratio_2021

    dict_aux = {solution_2020: ('email 2020', df_2020), solution_2021: ('email 2021', df_2021)}
    
    # Agregamos los correos
    for key, value in dict_aux.items():
        try:
            df_classes.at[row, value[0]] = value[1][value[1]['Nombre'] == key]['Mail'].values[0].strip()
        except:
            pass
    
# This line orders our dataframe columns as wanted
df_classes = df_classes[[cols[0]] + cols[-6:] + cols[1:-6]]

# Finally, we update our dictionary, generate a new Excel file, and display our edited dataframe
dict_final = {'students2020': df_students01, 'students2021': df_students02, 'classes': df_classes}

writer = pd.ExcelWriter('esperemos_que_funcione.xlsx', engine='xlsxwriter')

for df_name, df in dict_final.items():
    df.to_excel(writer, sheet_name=df_name)
    
writer.save()

display(df_classes)

100%|██████████| 647/647 [00:08<00:00, 73.33it/s]


,Nombre,name 2020,ratio 2020,email 2020,name 2021,ratio 2021,email 2021,Grupo,Ciclo,Días,...,Clase 3,Clase 4,Clase 5,Clase 6,Clase 7,Clase 8,Comentario,Velo,Interpretación,Ritmología
0,Cata Abarca,Catalina Abarca,0.846154,None,Catalina Abarca Aguilar,0.647059,catalina.abarca.13@gmail.com,Grupo 1,Ciclo 1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Marcela Aguilar,Marcela Aguilar,1,None,Marcela Aguilar,1,aguilarguzman@gmail.com,Grupo 1,Ciclo 1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Paulina Valenzuela,Paulina Valenzuela,0.972973,None,Paulina Valenzuela,0.972973,pvalenzuela33@gmail.com,Grupo 1,Ciclo 1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Khaterine Mardones,Katherine Mardones,0.944444,None,Katherine Mardones,0.944444,kmardones@udec.cl,Grupo 1,Ciclo 1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Mónica Neumann,Mónica Neumann,1,None,Mónica Neumann,1,None,Grupo 1,Ciclo 1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
642,Nicole Constenla,Nicole Romo Contreras,0.578947,nicolcita.love.17@gmail.com,Nicole Constenla Jofre,0.842105,nico.constenla@hotmail.com,Workshop,Ciclo 2,Sa,...,2020-02-20 00:00:00,2020-02-27 00:00:00,NaN,NaN,NaN,NaN,NaN,Si,Si,Si
643,Joann Donoso,Paula Troncoso,0.538462,paulatroncosokirsten@gmail.com,Joann Donoso Lemus,0.8,joanndonoso@hotmail.com,Workshop,Ciclo 2,Sa,...,2020-02-20 00:00:00,2020-02-27 00:00:00,NaN,NaN,NaN,NaN,NaN,Si,NaN,NaN
644,Javiera Bolados,Javiera Olid,0.740741,javi.olid.s@gmail.com,Javiera Bolados,1,javiera.bolados@gmail.com,Workshop,Ciclo 2,Sa,...,2020-02-20 00:00:00,2020-02-27 00:00:00,NaN,NaN,NaN,NaN,NaN,Si,Si,Si
645,Ana de la Fuente,Victoria Alejandra Urra Fuentes,0.5,victoriaurraf@gmail.com,Ana de la Fuente Parra,0.842105,ritmo8@hotmail.com,Workshop,Ciclo 2,Sa,...,2020-02-20 00:00:00,2020-02-27 00:00:00,NaN,NaN,NaN,NaN,NaN,Si,Si,NaN


In [8]:
for key in dict_final.keys():
    print(key)

students2020
students2021
classes


In [116]:
display(dict_final['students2021'])

,Nombre,Orden,Grupo Inicial,Curso Actual,Teléfono,Mail,Instagram,País/ciudad/comuna,online o grabaciónes,Estado,Fecha de nacimiento,Edad,Ocupación,"Intereses, hijos, cometarios extras.",Fecha de pago de la inscripción,Fecha de inscripción a la ficha,Cantidad de meses activa,Nombre 2020,Name match %
0,Catalina Abarca Aguilar,1,1,NaN,NaN,catalina.abarca.13@gmail.com,coabarca,Peñalolén,online,Alumna Activa,1900-01-23,23.0,Estudiante de Educación en párbulos.,Mamá e hija,NaT,NaT,NaN,Catalina Abarca,0.789474
1,Marcela Aguilar,2,1,NaN,NaN,aguilarguzman@gmail.com,marce.aguilar.g,Peñalolén,NaN,Alumna Inactiva,1900-02-17,48.0,Periodista,NaN,NaT,NaT,NaN,Marcela Aguilar,1.000000
2,Mónica Neumann,3,1,NaN,NaN,NaN,NaN,NaN,NaN,Alumna Inactiva,NaT,NaN,NaN,NaN,NaT,NaT,NaN,Mónica Neumann,1.000000
3,Paulina Valenzuela,4,1,NaN,NaN,pvalenzuela33@gmail.com,gerlachpaulina,"Santiago, Las condes",online,Alumna Inactiva,1900-03-01,61.0,Periodista,"mamá. Hace zumba, ballroom.",NaT,NaT,NaN,Paulina Valenzuela,1.000000
4,Katherine Mardones,5,1,NaN,NaN,kmardones@udec.cl,katymluna,"Hualpén, Concepción",online,Alumna Activa,1900-01-30,30.0,NaN,mamá,NaT,NaT,NaN,Katherine Mardones,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,Soraya Farah,223,técnica 2,técnica 2,56961860262,sora189@hotmail.com,sor.farah,Ñuñoa,online,Alumna Activa,1900-01-31,31.0,NaN,NaN,2021-03-08,NaT,NaN,NaN,NaN
221,Aracely Chicao Fica,224,técnica 1,técnica 1,569 65336124,chicaoary@gmail.com,Ary knowles,"Perquenco, temuco",online,Alumna Activa,1900-01-26,26.0,NaN,NaN,2021-03-08,NaT,NaN,NaN,NaN
222,Ganadora: Bárbara Arévalo,225,Técnica 3,Técnica 3,56940721927,barbara.chile94@gmail.com,barbaratravels_,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN
223,Ganadora: Constanza Cisternas,226,técnica 2,técnica 2,56932437793,c.cisternas.na@gmail.com,consttanttin_e,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN


<h2>Preparación de la hoja de clases</h2>

In [124]:
df_test = dict_final['classes'].copy(deep=True)
df_2021 = dict_final['students2021'].copy(deep=True)

cols = [col for col in df_test]
df_test['Correos'] = None

for row in range(df_test.shape[0]):
    current_row = df_test.loc[row]
    name = current_row['name 2021']
    
    # En las siguientes líneas corregimos el error en la fecha del workshop
    if (current_row['Grupo'] == 'Workshop') and (current_row['Días'] == 'Sa'):
        df_test.at[row, 'Inicio'] = current_row['Inicio'].replace(year=2021)
        df_test.at[row, 'Termino'] = current_row['Termino'].replace(year=2021)
        
    # Agregamos los correos
    try:
        df_test.at[row, 'Correos'] = df_2021[df_2021['Nombre'] == name]['Mail'].values[0].strip()
    except:
        pass
        
# Utilizando las columnas dejamos la llave al comienzo
df_test = df_test[[cols[-1]] + ['Correos'] + cols[1:-1] + [cols[0]]]

# Eliminamos los grupos que no iniciaron en 2021, ordenamos por fecha y se borran duplicados
df_test = df_test[(df_test['Inicio'].dt.year == 2021) & (df_test['Grupo'] != 'Workshop')]
df_test = df_test.sort_values(by='Inicio', ascending=False)
#df_test = df_test.drop_duplicates(subset='name 2021', keep='last')

df_test.to_excel('mailing_with_duplicates.xlsx')
display(df_test)

,name 2021,Correos,Grupo,Ciclo,Días,Inicio,Termino,Pago,Clase 1,Clase 2,...,Clase 4,Clase 5,Clase 6,Clase 7,Clase 8,Comentario,Velo,Interpretación,Ritmología,Nombre
517,Yasna Muñoz Vargas,yasna.munoz.vargas@gmail.com,Técnica 1,Ciclo 3,Ma - Ju,2021-03-09,2021-03-30 00:00:00,25000.0,2021-03-09,2021-03-16 00:00:00,...,2021-03-30 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yasna Muñoz
574,Paulina Díaz,diaz.lopez.paulina@gmail.com,Técnica 2,Ciclo 3,Ju,2021-03-09,2021-03-30 00:00:00,18000.0,2021-03-09,2021-03-16 00:00:00,...,2021-03-30 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Paulina Díaz
518,Alejandra Espinosa Pino,ale.espinosa92@gmail.com,Técnica 1,Ciclo 3,Ma,2021-03-09,2021-03-30 00:00:00,18000.0,2021-03-09,2021-03-16 00:00:00,...,2021-03-30 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alejandra Espinosa
559,Mariela Presman,marielapresman@gmail.com,Técnica 2,Ciclo 3,Ma - Ju,2021-03-09,NaN,25000.0,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mariela Presman
560,Tania Varelo Pizarro,nia.varelo.tv@gmail.com,Técnica 2,Ciclo 3,Ju,2021-03-09,44285,18000.0,2021-03-09,44271,...,44285,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tania Varelo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
557,Bianca Navarrete Brandt,bianca.nav03@gmail.com,Técnica 2,Ciclo 1,Mi y Vi,2021-01-06,2021-01-27 00:00:00,25000.0,2021-01-06,2021-01-13 00:00:00,...,2021-01-27 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bianca Navarrete
462,Yulieth holguin,Yuliholguin24@hotmail.com,Técnica 1,Ciclo 1,Mi,2021-01-06,2021-01-27 00:00:00,18000.0,2021-01-06,2021-01-13 00:00:00,...,2021-01-27 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yulieth holguin
461,Macarena Dib,macadib@hotmail.com,Técnica 1,Ciclo 1,Mi,2021-01-06,2021-01-27 00:00:00,12500.0,2021-01-06,2021-01-13 00:00:00,...,2021-01-27 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Macarena Dib
460,Ruth Dorat Coronado,ruthdorat.to@gmail.com,Técnica 1,Ciclo 1,Mi,2021-01-06,2021-01-27 00:00:00,18000.0,2021-01-06,2021-01-13 00:00:00,...,2021-01-27 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ruth Dorat


<h1 align='center'>Hasta aquí código funcional</h1>

<h2>Comparisons as dictionaries</h2>

In [143]:
df_students01 = df_main['alumnas2020'].copy(deep=True)
df_students02 = df_main['alumnas2021'].copy(deep=True)


def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()


def gen_similar(title01, list01, title02, list02, title_ratio):
    dict_similars = {title01: [], title_ratio: [], title02: []}
    
    for word in list01:
        ratio = 0

        for spiegel in list02:
            if ratio < max(ratio, similar(word, spiegel)):
                ratio = similar(word, spiegel)
                solution = {title01: word, title_ratio: ratio, title02: spiegel}
                
        for key in dict_similars.keys():
                dict_similars[key].append(solution[key])

    return(dict_similars)


list01 = df_students01['Nombre'].to_list()
list02 = df_students02['Nombre'].to_list()
dict_similars = gen_similar('2020 names', list01, '2021 names', list02, '2020 to 2021 names')

Heidelinde Matzke Madrid


In [167]:
df_classes = df_main['clases'].copy(deep=True)


def gen_similar(list01, list02):
    list_similars = []
    
    for word in list01:
        ratio = 0

        for spiegel in list02:
            if ratio < max(ratio, similar(word, spiegel)):
                ratio = similar(word, spiegel)
                solution = (word, ratio, spiegel)

        list_similars.append(solution)

    return(list_similars)


list01 = df_students02['Nombre'].to_list()
list02 = df_classes['Nombre'].to_list()
list_similars = gen_similar(list01, list02)

print(df_classes['Nombre'][0], list_similars)

for row in range(df_classes.shape[0]):
    name_2021 = df_classes.at[row, 'Nombre']
    
    for i, col in enumerate(['Nombre 2021', 'Name match %']):
        value = [entry[i] for entry in list_similars if entry[-1] == name_2021]

        if len(value) != 0:
            df_classes.at[row, col] = value[0]
            
display(df_classes)

Cata Abarca [('Catalina Abarca Aguilar', 0.7894736842105263, 'Catalina Abarca'), ('Marcela Aguilar', 1.0, 'Marcela Aguilar'), ('Mónica Neumann', 1.0, 'Mónica Neumann'), ('Paulina Valenzuela ', 0.972972972972973, 'Paulina Valenzuela'), ('Katherine Mardones', 1.0, 'Katherine Mardones'), ('Claudia Alvarado', 1.0, 'Claudia Alvarado'), ('Fabiola Saez', 1.0, 'Fabiola Saez'), ('Valentina Rivera', 1.0, 'Valentina Rivera'), ('Andrea Gutiérrez Martínez ', 0.7619047619047619, 'Andrea Gutiérrez'), ('Carla Guerrero Silva', 0.8235294117647058, 'Carla Guerrero'), ('Carolina Atala Salim-Hanna', 0.7, 'Carolina Atala'), ('Natalia landeros', 1.0, 'Natalia landeros'), ('Fernanda Landeros Pérez', 1.0, 'Fernanda Landeros Pérez'), ('Gigi', 1.0, 'Gigi'), ('Antonia Romero', 1.0, 'Antonia Romero'), ('Antonia Brito', 0.7407407407407407, 'Antonia Romero'), ('Javiera Olid', 0.8571428571428571, 'Javi Olid'), ('Cintia Sanocky', 1.0, 'Cintia Sanocky'), ('Valentina Loiza', 1.0, 'Valentina Loiza'), ('Valeria Pontillo',

,Nombre,Grupo,Ciclo,Días,Inicio,Termino,Pago,Clase 1,Clase 2,Clase 3,...,Clase 5,Clase 6,Clase 7,Clase 8,Comentario,Velo,Interpretación,Ritmología,Nombre 2021,Name match %
0,Cata Abarca,Grupo 1,Ciclo 1,NaN,2020-04-01,2020-04-22,18000.0,NaT,NaT,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Marcela Aguilar,Grupo 1,Ciclo 1,NaN,2020-04-01,2020-04-22,18000.0,NaT,NaT,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Marcela Aguilar,1.000000
2,Paulina Valenzuela,Grupo 1,Ciclo 1,NaN,2020-04-01,2020-04-22,18000.0,NaT,NaT,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Paulina Valenzuela,0.972973
3,Khaterine Mardones,Grupo 1,Ciclo 1,NaN,2020-04-01,2020-04-22,18000.0,NaT,NaT,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Mónica Neumann,Grupo 1,Ciclo 1,NaN,2020-04-01,2020-04-22,18000.0,NaT,NaT,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mónica Neumann,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596,Nicole Constenla,Workshop,Ciclo 2,Sa,2020-02-06,2020-02-27,35000.0,2020-02-06,2020-02-13,2020-02-20,...,NaN,NaN,NaN,NaN,NaN,Si,Si,Si,Nicole Constenla Jofre,0.842105
597,Joann Donoso,Workshop,Ciclo 2,Sa,2020-02-06,2020-02-27,15000.0,2020-02-06,2020-02-13,2020-02-20,...,NaN,NaN,NaN,NaN,NaN,Si,NaN,NaN,Joann Donoso Lemus,0.800000
598,Javiera Bolados,Workshop,Ciclo 2,Sa,2020-02-06,2020-02-27,35000.0,2020-02-06,2020-02-13,2020-02-20,...,NaN,NaN,NaN,NaN,NaN,Si,Si,Si,Javiera Bolados,1.000000
599,Ana de la Fuente,Workshop,Ciclo 2,Sa,2020-02-06,2020-02-27,40000.0,2020-02-06,2020-02-13,2020-02-20,...,NaN,NaN,NaN,NaN,NaN,Si,Si,NaN,Ana de la Fuente Parra,0.842105
